## Testes iniciais

In [2]:
import requests

response = requests.get("http://api.open-notify.org/astros.json")
print(response)

<Response [200]>


In [6]:
#response.content() # Return the raw bytes of the data payload
#response.text() # Return a string representation of the data payload
response.json() # This method is convenient when the API returns JSON

{'args': {},
 'data': '',
 'files': {},
 'form': {'key': 'value'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '9',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.25.1',
  'X-Amzn-Trace-Id': 'Root=1-619d309d-5914ce0105405ed66647b9c4'},
 'json': None,
 'origin': '189.123.226.249',
 'url': 'https://httpbin.org/post'}

In [3]:
query = {'lat':'45', 'lon':'180'}
response = requests.get('http://api.open-notify.org/iss-pass.json', params=query)
print(response.json())

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1637688230, 'latitude': 45.0, 'longitude': 180.0, 'passes': 5}, 'response': [{'duration': 308, 'risetime': 1637721912}, {'duration': 632, 'risetime': 1637727506}, {'duration': 655, 'risetime': 1637733298}, {'duration': 629, 'risetime': 1637739152}, {'duration': 647, 'risetime': 1637744989}]}


In [4]:
# Create a new resource
response = requests.post('https://httpbin.org/post', data = {'key':'value'})
# Update an existing resource
requests.put('https://httpbin.org/put', data = {'key':'value'})

<Response [200]>

# Teste do servidor

## Visita servidor

#### Thiago visita servidor

In [3]:
import requests

channel = 'channel'
print(requests.get('http://127.0.0.1:5000/users', params={'channel': channel, 'user': 'Thiago', 'request': 'visit'}).json())

401


#### Schinda visita servidor, mas não passa nome como parametro

In [12]:
print(requests.get('http://127.0.0.1:5000/users', params={'request': 'visit'}).json())

{'message': {'channel': 'Missing required parameter in the JSON body or the post body or the query string'}}


#### Schinda Visita servidor

In [13]:
print(requests.get('http://127.0.0.1:5000/users', params={'channel': 'channel2', 'user': 'Schinda', 'request': 'visit'}).json())

200


## Cadastra enquete

#### Thiago cria enquete

In [14]:
import requests

print(requests.post('http://127.0.0.1:5000/users', data={'user': 'Thiago',
                                                         'enquete': 'Bonde de Floripa',
                                                         'local': 'Florianopolis',
                                                         'limite': '2021-11-20',
                                                         'votos': "['2021-11-19','2021-11-20','2021-11-21']"}
                   ))

<Response [200]>


#### Schinda cria enquete

In [15]:
print(requests.post('http://127.0.0.1:5000/users', data={'user': 'Schinda',
                                                         'enquete': 'Carnava',
                                                         'local': 'Guaratuba',
                                                         'limite': '2021-12-20',
                                                         'votos': "['2022-02-19','2022-02-20','2022-02-21']"}
                   ))

<Response [200]>


## Consulta enquete

#### Thiago acessa enquete que não existe

In [16]:
print(requests.get('http://127.0.0.1:5000/users', params={'user': 'Thiago', 'enquete': 'asdf', 'request': 'get info'}).json())

404


#### Schinda acessa enquete que não está cadastrado

In [17]:
print(requests.get('http://127.0.0.1:5000/users', params={'user': 'Schinda', 'enquete': 'Bonde de Floripa', 'request': 'get info'}).json())

401


#### Thiago acessa enquete que está cadastrado

In [18]:
print(requests.get('http://127.0.0.1:5000/users', params={'user': 'Thiago', 'enquete': 'Bonde de Floripa', 'request': 'get info'}).json())

{'request': {'limite': '2021-11-20', 'local': 'Florianopolis', 'status': 'Em andamento', 'votantes': ['Thiago'], 'votos': {'2021-11-19': 0, '2021-11-20': 0, '2021-11-21': 0}}}


## Cadastra voto em enquete

#### Schinda vota em enquete q não existe

In [19]:
print(requests.put('http://127.0.0.1:5000/users', params={'user': 'Schinda', 
                                                          'enquete': 'Bonde de Flo',
                                                          'voto': '2021-11-19'}).json())

404


#### Schinda vota em enquete com data não permitida de voto

In [20]:
print(requests.put('http://127.0.0.1:5000/users', params={'user': 'Schinda', 
                                                          'enquete': 'Bonde de Floripa',
                                                          'voto': '2020-01-01'}).json())

404


#### Schinda vota em enquete certinho

In [21]:
print(requests.put('http://127.0.0.1:5000/users', params={'user': 'Schinda', 
                                                          'enquete': 'Bonde de Floripa',
                                                          'voto': '2021-11-19'}).json())

200


## Consulta enquete

#### Schinda acessa enquete que está cadastrado após voto

In [22]:
print(requests.get('http://127.0.0.1:5000/users', params={'user': 'Schinda', 'enquete': 'Bonde de Floripa', 'request': 'get info'}).json())

{'request': {'limite': '2021-11-20', 'local': 'Florianopolis', 'status': 'Em andamento', 'votantes': ['Thiago', 'Schinda'], 'votos': {'2021-11-19': 1, '2021-11-20': 0, '2021-11-21': 0}}}


#### Código python para testar SSE do servidor

In [23]:
import json
import pprint
import requests
import sseclient # sseclient-py

# change the name for each client
input_json = {'name':'Alice'}
#input_json = {'name':'Bob'}
#input_json = {'name':'Carol'}

url = 'http://127.0.0.1:5000/users'
stream_response = requests.get(url, params={'user': 'Thiago', 'request': 'visit'}, stream=True)

client = sseclient.SSEClient(stream_response)

# Loop forever (while connection "open")
for event in client.events():
    print ("got a new event from server")
    pprint.pprint(event.data)

InvalidURL: Failed to parse: <Response [200]>